## Decision Tree

CART : Classification and Regression Trees

Daha önceki dökümanlarda decision tree algoritmasının regression ile nasıl yapıldığını anlatmıştım. 

<br><br>
<img src="d1.png" style="width:400px;height:300px"/>

Yukarıda gördüğümüz üzere mor ve mavi şeklinde iki sınıf var. Decision tree algoritması ile sınıflandırma işleminde veriler parçalanır, split'lere ayrılır. 

<br><br>
<img src="d2.png" style="width:400px;height:300px"/>

Şekilde gördüğünüz split, x1 ve x2 feature'lerine göre görselleştirilen veri noktalarını mor ve mavi sınıf en iyi ayrılacak şekilde aralarına bir sınır çizilmekte. Matematiksel olarak amaçlanan düşük entropi değeridir.

Split'ler mavi ve mor kategorilerini en iyi ayıracak şekilde seçilir. Şekilde gördüğünüz üzere split1 pek iyi bir seçim olmamıştır. Çünkü çizginin bir tarafında hem mavi hem de mor sınıfa ait noktalar bulunmakta. Dolayısıyla sınıfları doğru ayırabilmek için başka split'lerde kullanmalıyız.

<br><br>
<img src="d3.png" style="width:400px;height:300px"/>

split2 ile veri seti gayet güzel bir şekilde ayrılmış oluyor. Bu çizgilerin hizzasına karşılık gelen değerler şöyle olsun:

<br><br>
<img src="d5.png" style="width:400px;height:300px"/>

Yukarıdaki ayrıma göre ağaç şu şekilde olur:

<br><br>
<img src="d4.png" style="width:400px;height:300px"/>

Ayrılan her bir bölge olabildiğince sadece tek bir sınıfa ait noktalardan oluşacak şekilde split'ler seçilir.

İyi huylu ve kötü huylu tümörlerden oluşan veri setini kullanalım.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("data.csv")

In [3]:
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

Kullanmayacağımız feature'leri çıkaralım. 

In [5]:
data.drop(["id","Unnamed: 32"],axis=1, inplace=True)  
#id ve Unnamed: 32 sütunlarını tamamen drop edeceğimiz için yani tüm sütunu drop edeceğimiz için axis=1 yapılır.
#Kaldırma işlemini inplace=True parametresi ile veri setine kaydederiz.

diagnosis feataure'sini class olarak kullanacağız. Ama bu sütun string'ten oluşuyor fakat class label'larının categorical veya int tipinde olması gerekir sklearn kütüphanesinde kullanabilmemiz için. Dolayısıyla kötü huylu tümörleri 1, iyi huylu tümörleri 0 olacak şekilde int tipine çevirelim.

In [6]:
data.diagnosis = [1 if each == "M" else 0 for each in data.diagnosis]

In [7]:
y = data.diagnosis.values
x_data = data.drop(["diagnosis"],axis=1)  #diagnosis çıkarılır ve geri kalan feature'ler x_data feature'sine atanır.

In [8]:
#Normalization
x = (x_data - np.min(x_data))/(np.max(x_data)-np.min(x_data))

In [9]:
#train test split

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.15,random_state=42)

In [10]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(x_train,y_train)

dt.score(x_test,y_test)

0.9186046511627907

%91 oranında doğruluk değeri elde ettik. Yani x_test'te bulunan 86 sample'ın %91 'i doğru tahmin edilmiştir.